# Maryland Automotive Car Industry

### Import Packages

In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

### Import Functions

In [2]:
import re

def camel_to_snake(column_names):
    def convert(name):
        name = name.strip()
        name = re.sub(r'\s+', '_', name).lower()
        name = re.sub(r'([a-z])([A-Z])', r'\1_\2', name).lower()
        return name
    
    return [convert(name) for name in column_names]

# use this function to create dataframes with appropriate column names
def rename_columns_to_snake_case(df):
    df.columns = camel_to_snake(df.columns)
    return df


## Car Sales Data

### MVA Vehicle Sales Counts Data Preparation
The 'sales_data' is from the Maryland State Government which released the data related to new and used car sales and value of car sales from 2002 to the foreseeable future. 

This information can be used for forecasting and understanding the direction of the automotive industry post Covid-19 which rocked global economies.

The full information regarding the MVA Vehicle Sales dataset can be found in the bibliography of the datasets in 'reports'.

In [3]:
# import the sales data
sales_data = pd.read_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Maryland Automotive Industry/raw_datasets/cars_datasets/MVA_Vehicle_Sales_Counts_by_Month_for_Calendar_Year_2002_through_December_2023.csv')
sales_data

,Year,Month,New,Used,Total Sales New,Total Sales Used
0,2002,JAN,31106,49927,755015820,386481929
1,2002,FEB,27520,50982,664454223,361353242
2,2002,MAR,34225,58794,805666244,419385387
3,2002,APR,36452,59817,846368297,433061150
4,2002,MAY,37359,60577,855005784,442569410
...,...,...,...,...,...,...
259,2023,AUG,25876,52725,1222581892,908454060
260,2023,SEP,23892,45386,1134437699,744676584
261,2023,OCT,23775,45473,1122680147,740582533
262,2023,NOV,22720,42260,1062465105,694190564


In [4]:
# rename the columns to be in snake case
sales_data = rename_columns_to_snake_case(sales_data)
sales_data

,year,month,new,used,total_sales_new,total_sales_used
0,2002,JAN,31106,49927,755015820,386481929
1,2002,FEB,27520,50982,664454223,361353242
2,2002,MAR,34225,58794,805666244,419385387
3,2002,APR,36452,59817,846368297,433061150
4,2002,MAY,37359,60577,855005784,442569410
...,...,...,...,...,...,...
259,2023,AUG,25876,52725,1222581892,908454060
260,2023,SEP,23892,45386,1134437699,744676584
261,2023,OCT,23775,45473,1122680147,740582533
262,2023,NOV,22720,42260,1062465105,694190564


In [5]:
# rename columns to be more appropriate
sales_data.rename(columns={'new':'new_cars_sold',
                           'used':'used_cars_sold',
                           'total_sales_new':'new_value_cars_sold',
                           'total_sales_used':'used_value_cars_sold'},inplace=True)

In [6]:
# dictionary mapping month names to numbers
month_mapping = {
    "JAN": 1,
    "FEB": 2,
    "MAR": 3,
    "APR": 4,
    "MAY": 5,
    "JUN": 6,
    "JUL": 7,
    "AUG": 8,
    "SEP": 9,
    "OCT": 10,
    "NOV": 11,
    "DEC": 12
}

# replace month names with numbers in the specified column
sales_data['month'] = sales_data['month'].replace(month_mapping)

In [7]:
# create a 'date' column
date_column = pd.to_datetime(sales_data[['year', 'month']].assign(DAY=1), format='%Y-%m-%d')

# insert the column into the first column index
sales_data.insert(0,'date',date_column)

sales_data.set_index('date',inplace=True)

# drop the old columns
sales_data.drop(columns={'year','month'},inplace=True)

#### Sales Data Column Creation
Create the following columns

- total_cars_sold : find the total number of cars sold
- total_value_cars_sold : find the total value of cars sold

In [8]:
# get the total cars sold and insert at appropriate position
total_cars_sold = sales_data['new_cars_sold'] + sales_data['used_cars_sold']
sales_data.insert(0,'total_cars_sold',total_cars_sold)

# get the total value of cars sold and insert at appropriate position
total_value_cars_sold = sales_data['new_value_cars_sold'] + sales_data['used_value_cars_sold']
sales_data.insert(3,'total_value_cars_sold',total_value_cars_sold)

sales_data

,total_cars_sold,new_cars_sold,used_cars_sold,total_value_cars_sold,new_value_cars_sold,used_value_cars_sold
date,,,,,,
2002-01-01,81033,31106,49927,1141497749,755015820,386481929
2002-02-01,78502,27520,50982,1025807465,664454223,361353242
2002-03-01,93019,34225,58794,1225051631,805666244,419385387
2002-04-01,96269,36452,59817,1279429447,846368297,433061150
2002-05-01,97936,37359,60577,1297575194,855005784,442569410
...,...,...,...,...,...,...
2023-08-01,78601,25876,52725,2131035952,1222581892,908454060
2023-09-01,69278,23892,45386,1879114283,1134437699,744676584
2023-10-01,69248,23775,45473,1863262680,1122680147,740582533


In [9]:
# save the data to 'prepared_datasets'
sales_data.to_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Maryland Automotive Industry/preprocessed_datasets/car_datasets/sales_data.csv')

### Electric Car Vehicle Data

In [10]:
# read in the electric car vehicle data
ecar_sales_data = pd.read_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Maryland Automotive Industry/raw_datasets/cars_datasets/MD_MDOT_MVA_Electric_and_Plug-in_Hybrid_Vehicle_Registrations_by_Zip_Code_as_of_Each_Month_End_from_July_2020_to_April_2024.csv')
ecar_sales_data

,Year_Month,Fuel_Category,Zip_Code,Count
0,2020/07,Electric,19973,1
1,2020/07,Electric,20601,21
2,2020/07,Electric,20602,26
3,2020/07,Electric,20603,54
4,2020/07,Electric,20607,35
...,...,...,...,...
58297,2024/04,Plug-In Hybrid,95130,1
58298,2024/04,Plug-In Hybrid,95628,1
58299,2024/04,Plug-In Hybrid,97703,1
58300,2024/04,Plug-In Hybrid,98045,1


In [11]:
# rename the columns to industry standards
ecar_sales_date = rename_columns_to_snake_case(ecar_sales_data)
ecar_sales_data

,year_month,fuel_category,zip_code,count
0,2020/07,Electric,19973,1
1,2020/07,Electric,20601,21
2,2020/07,Electric,20602,26
3,2020/07,Electric,20603,54
4,2020/07,Electric,20607,35
...,...,...,...,...
58297,2024/04,Plug-In Hybrid,95130,1
58298,2024/04,Plug-In Hybrid,95628,1
58299,2024/04,Plug-In Hybrid,97703,1
58300,2024/04,Plug-In Hybrid,98045,1


In [12]:
# convert the current object dtype date form into datetime format
ecar_sales_data['date'] = ecar_sales_data['year_month'].apply(lambda x: datetime.strptime(x, '%Y/%m'))

# set the index to date
ecar_sales_data.set_index('date',inplace=True)

# drop the original data
ecar_sales_data.drop(columns='year_month',inplace=True)

ecar_sales_data

,fuel_category,zip_code,count
date,,,
2020-07-01,Electric,19973,1
2020-07-01,Electric,20601,21
2020-07-01,Electric,20602,26
2020-07-01,Electric,20603,54
2020-07-01,Electric,20607,35
...,...,...,...
2024-04-01,Plug-In Hybrid,95130,1
2024-04-01,Plug-In Hybrid,95628,1
2024-04-01,Plug-In Hybrid,97703,1


In [13]:
# drop the 'in' and replace with 'In'
# this will reduce the redundancy in recording of data
ecar_sales_data['fuel_category'] = ecar_sales_data['fuel_category'].apply(lambda x: x.replace('in', 'In'))

In [14]:
# convert the data to be sorted by both 'date' and 'fuel_category'
ecar_sales_data = ecar_sales_data.groupby(['date', 'fuel_category'])['count'].sum().reset_index()
ecar_sales_data

,date,fuel_category,count
0,2020-07-01,Electric,15192
1,2020-07-01,Plug-In Hybrid,11026
2,2020-08-01,Electric,15578
3,2020-08-01,Plug-In Hybrid,11153
4,2020-09-01,Electric,15889
...,...,...,...
87,2024-02-01,Plug-In Hybrid,29622
88,2024-03-01,Electric,69529
89,2024-03-01,Plug-In Hybrid,30216
90,2024-04-01,Electric,71715


In [15]:
# pivot the table so that the columns are based on the categorical variable of 'fuel_category'
ecar_sales_data = ecar_sales_data.pivot(index='date', columns='fuel_category', values='count')
ecar_sales_data.head()

fuel_category,Electric,Plug-In Hybrid
date,,
2020-07-01,15192,11026
2020-08-01,15578,11153
2020-09-01,15889,11297
2020-10-01,16549,11453
2020-11-01,16906,11669


## Economic Data

### Inflation Rate in Maryland

In [16]:
inflation_rate = pd.read_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Maryland Automotive Industry/raw_datasets/economic_datasets/CPIAUCSL.csv')
inflation_rate

,DATE,CPIAUCSL
0,1995-01-01,150.500
1,1995-02-01,150.900
2,1995-03-01,151.200
3,1995-04-01,151.800
4,1995-05-01,152.100
...,...,...
348,2024-01-01,309.685
349,2024-02-01,311.054
350,2024-03-01,312.230
351,2024-04-01,313.207


In [17]:
# change the columns of the dataframe
inflation_rate = rename_columns_to_snake_case(inflation_rate)

# convert date to datetime
inflation_rate['date'] = pd.to_datetime(inflation_rate['date'])

inflation_rate.set_index('date',inplace=True)

inflation_rate

,cpiaucsl
date,
1995-01-01,150.500
1995-02-01,150.900
1995-03-01,151.200
1995-04-01,151.800
1995-05-01,152.100
...,...
2024-01-01,309.685
2024-02-01,311.054
2024-03-01,312.230


In [18]:
inflation_rate = inflation_rate[(inflation_rate.index>='2002-01-01')&(inflation_rate.index<'2024-01-01')]

inflation_rate

,cpiaucsl
date,
2002-01-01,177.700
2002-02-01,178.000
2002-03-01,178.500
2002-04-01,179.300
2002-05-01,179.500
...,...
2023-08-01,306.187
2023-09-01,307.288
2023-10-01,307.531


In [19]:
inflation_rate.to_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Maryland Automotive Industry/preprocessed_datasets/economic_datasets/inflation_rate.csv')

### Unemployment Rate in Maryland

In [20]:
# import unemployment rate data 
unemployment_rate = pd.read_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Maryland Automotive Industry/raw_datasets/economic_datasets/MDUR.csv')

unemployment_rate.rename(columns={'MDUR':'Unemployment Rate'},inplace=True)

unemployment_rate = rename_columns_to_snake_case(unemployment_rate)

unemployment_rate.set_index('date',inplace=True)

unemployment_rate

,unemployment_rate
date,
1976-01-01,6.4
1976-02-01,6.4
1976-03-01,6.4
1976-04-01,6.4
1976-05-01,6.5
...,...
2024-01-01,2.3
2024-02-01,2.4
2024-03-01,2.5


In [21]:
# unemployment for the appropriate time period
unemployment_rate = unemployment_rate[(unemployment_rate.index >='2002-01-01') & (unemployment_rate.index<'2024-01-01')]
unemployment_rate

,unemployment_rate
date,
2002-01-01,5.0
2002-02-01,5.0
2002-03-01,5.0
2002-04-01,4.9
2002-05-01,4.9
...,...
2023-08-01,2.1
2023-09-01,2.2
2023-10-01,2.3


In [22]:
unemployment_rate.to_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Maryland Automotive Industry/preprocessed_datasets/economic_datasets/unemployment.csv')

### GDP for Maryland

In [23]:
# gdp data
gdp = pd.read_excel('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Maryland Automotive Industry/raw_datasets/economic_datasets/gdp.xlsx')
gdp = gdp.T
gdp.reset_index(inplace=True)
gdp.columns = ['date','gdp']
gdp.set_index('date',inplace=True)

gdp

,gdp
date,
2002-01-01,217001.4
2003-01-01,226803.4
2004-01-01,243449.8
2005-01-01,261795.3
2006-01-01,273791.0
2007-01-01,282618.2
2008-01-01,292908.3
2009-01-01,298282.8
2010-01-01,313930.5


In [24]:
gdp.to_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Maryland Automotive Industry/preprocessed_datasets/economic_datasets/gdp.csv')

### Interest Rates

In [25]:
interest_rates = pd.read_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Maryland Automotive Industry/raw_datasets/economic_datasets/FEDFUNDS.csv')
interest_rates = rename_columns_to_snake_case(interest_rates)

interest_rates

,date,fedfunds
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83
...,...,...
835,2024-02-01,5.33
836,2024-03-01,5.33
837,2024-04-01,5.33
838,2024-05-01,5.33


In [26]:
interest_rates = interest_rates[(interest_rates['date']>='2002-01-01')&(interest_rates['date']<'2024-01-01')]
interest_rates.set_index('date',inplace=True)
interest_rates

,fedfunds
date,
2002-01-01,1.73
2002-02-01,1.74
2002-03-01,1.73
2002-04-01,1.75
2002-05-01,1.75
...,...
2023-08-01,5.33
2023-09-01,5.33
2023-10-01,5.33


In [ ]:
interest_rates.to_csv('/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Maryland Automotive Industry/preprocessed_datasets/economic_datasets/interest_rates.csv')

## Socioeconomic Data